In [ ]:
# coding: utf-8
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
import random as rd

# Technical indicators from the ta library
from ta.momentum import RSIIndicator
from ta.trend import MACD, EMAIndicator, ADXIndicator, PSARIndicator
from ta.volatility import AverageTrueRange

#############################################
# MONTE CARLO SIMULATION FUNCTIONS
#############################################
def monte_carlo(data, testsize=0.5, simulation=100, **kwargs):
    df, test = train_test_split(data, test_size=testsize, shuffle=False, **kwargs)
    forecast_horizon = len(test)
    df = df.loc[:, ['Close']]
    returns = np.log(df['Close'].iloc[1:] / df['Close'].shift(1).iloc[1:])
    drift = returns.mean() - returns.var() / 2
    d = {}
    for counter in range(simulation):
        d[counter] = [df['Close'].iloc[0]]
        for i in range(len(df) + forecast_horizon - 1):
            sde = drift + returns.std() * rd.gauss(0, 1)
            temp = d[counter][-1] * np.exp(sde)
            d[counter].append(temp.item())
    std = float('inf')
    pick = 0
    for counter in range(simulation):
        temp = np.std(np.subtract(d[counter][:len(df)], df['Close']))
        if temp < std:
            std = temp
            pick = counter
    return forecast_horizon, d, pick

#############################################
# BTC TRADING STRATEGY FUNCTIONS & SETUP
#############################################

df = pd.read_csv('/content/BTC_2019_2023_1d.csv')
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df = df.dropna(subset=['datetime'])
df.set_index('datetime', inplace=True)
required_columns = ['open', 'high', 'low', 'close', 'signals']
for column in required_columns:
    if column not in df.columns:
        df[column] = 0 if column == 'signals' else np.nan

df.fillna(method='ffill', inplace=True)
df['HA_close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
df['HA_open'] = (df['open'].shift(1) + df['close'].shift(1)) / 2
df['HA_high'] = df[['high', 'HA_open', 'HA_close']].max(axis=1)
df['HA_low'] = df[['low', 'HA_open', 'HA_close']].min(axis=1)

df['RSI'] = RSIIndicator(close=df['HA_close'], window=14).rsi()
macd = MACD(close=df['HA_close'], window_slow=26, window_fast=12, window_sign=9)
df['MACD'] = macd.macd()
df['MACD_Signal'] = macd.macd_signal()
df['MACD_Hist'] = macd.macd_diff()
df['ema_10'] = EMAIndicator(close=df['HA_close'], window=10).ema_indicator()
df['ema_30'] = EMAIndicator(close=df['HA_close'], window=30).ema_indicator()
df['ema_200'] = EMAIndicator(close=df['HA_close'], window=150).ema_indicator()
df['ema_20'] = EMAIndicator(close=df['HA_close'], window=20).ema_indicator()
df['ATR'] = AverageTrueRange(high=df['HA_high'], low=df['HA_low'], close=df['HA_close'], window=14).average_true_range()
adx = ADXIndicator(high=df['HA_high'], low=df['HA_low'], close=df['HA_close'], window=14)
df['ADX'] = adx.adx()
df['ADX_Pos'] = adx.adx_pos()
df['ADX_Neg'] = adx.adx_neg()
psar = PSARIndicator(high=df['HA_high'], low=df['HA_low'], close=df['HA_close'], step=0.02, max_step=0.2)
df['PSAR'] = psar.psar()

rsi_buy_threshold = 40  # Increased to allow more trades
rsi_sell_threshold = 60  # Decreased to take profits earlier
adx_buy_threshold = 20  # Lowered to allow more entries
adx_sell_threshold = 30
transaction_cost = 0.00

in_position = False
position_type = 0
entry_price = 0
net_profit = 0
balance = 10000
btc_amount = 0
trade_count = 0
winning_trades = 0
losing_trades = 0
max_drawdown_value = 0
peak_balance = balance
signal_history = [0] * len(df)
trade_type = [None] * len(df)
balance_history = []

df['Close'] = df['close']
forecast_horizon, d, pick = monte_carlo(df[['Close']], testsize=0.5, simulation=100)
train_length = len(df) - forecast_horizon
historical_end_price = df['Close'].iloc[train_length - 1]
forecast_price = d[pick][-1]
MC_signal = np.sign(forecast_price - historical_end_price)
df['MC_forecast'] = forecast_price
df['MC_signal'] = MC_signal

for i, (index, row) in enumerate(df.iterrows()):
    current_price = row['close']
    rsi = row['RSI']
    ema_10 = row['ema_10']
    adx_val = row['ADX']
    psar_val = row['PSAR']

    buy_condition = (
        (rsi > rsi_buy_threshold and ema_10 >= row['ema_30']) and
        (adx_val > adx_buy_threshold) and
        (current_price > psar_val)
    )
    if buy_condition and not in_position and MC_signal > 0:
        entry_price = current_price
        btc_amount = balance / entry_price
        in_position = True
        position_type = 1
        trade_count += 1
        signal_history[i] = 1
        trade_type[i] = 'long'

    sell_condition_long = (
        (current_price <= psar_val) and
        (rsi < rsi_sell_threshold)
    )
    if in_position and position_type == 1 and sell_condition_long:
        balance = btc_amount * current_price
        net_profit += (current_price - entry_price) * btc_amount
        in_position = False
        signal_history[i] = -1
        trade_type[i] = 'exit'

    balance_history.append(balance)
    peak_balance = max(peak_balance, balance)
    drawdown = (peak_balance - balance) / peak_balance
    max_drawdown_value = max(max_drawdown_value, drawdown)

df['signals'] = signal_history
df['trade_type'] = trade_type
df['balance'] = balance_history
output_filename = f'strategy2BTCProfitUpdated.csv'
df.to_csv(output_filename, index=True)
print(f"File saved as {output_filename}")